In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import ruptures as rpt  # our package
import pickle


def determine_drive_state_changed(timeseries: list, n_bkps: int = 3) -> list:
    timeseries = np.array(timeseries)
    timeseries = timeseries / np.linalg.norm(timeseries)
    algo = rpt.KernelCPD(kernel="rbf").fit(timeseries)

    # Detect breakpoints
    breakpoints = algo.predict(n_bkps=n_bkps)
    driver_state_changed = [any(i >= bp for bp in breakpoints) for i in range(len(timeseries))]
    assert len(driver_state_changed) == len(timeseries)
    return driver_state_changed


## Driver change ensemble

In [ ]:
# Create template dataframe for submission
with open('../COOOL_benchmark/annotations_public.pkl', 'rb') as f:
    anns = pickle.load(f)
data = []
for video, video_data in anns.items():
    for frame, frame_data in video_data.items():
        data.append({
            'ID': f"{video}_{frame}",
            'video': video,
            'frame': frame,
        })
df = pd.DataFrame(data)


# Load predictions
df1 = pd.read_csv("../submissions/results_13122024_driver_of-bkp4.csv", index_col=0)
df1['frame'] = df1['ID'].apply(lambda x: int(x.split('_')[-1]) )
df1['video'] = df1['ID'].apply(lambda x: '_'.join(x.split('_')[:2]) )

df2 = pd.read_csv("../submissions/results_09122024_bboxsizes.csv")
df2 = df2[['ID', 'Driver_State_Changed']]
df2['frame'] = df2['ID'].apply(lambda x: int(x.split('_')[-1]) )
df2['video'] = df2['ID'].apply(lambda x: '_'.join(x.split('_')[:2]) )



# Create dict with index of first True value in Driver_State_Changed
data = defaultdict(list)
for i, group in df1.groupby(['video']):
    vals = group.reset_index(drop=True)['Driver_State_Changed']
    data[i].append(vals.idxmax())

for i, group in df2.groupby(['video']):
    vals = group.reset_index(drop=True)['Driver_State_Changed']
    data[i].append(vals.idxmax())



# Mean is the new True
idx = pd.DataFrame(data).T.mean(1).round().astype(int)
df_idx = idx.reset_index()
df_idx.columns = ['video', 'frame']
df_idx['change'] = True

df_new = pd.merge(df[['ID', 'video', 'frame']], df_idx, how='left', on=['video', 'frame'])
df_new['change'] = df_new['change'].fillna(False)
df_new['Driver_State_Changed'] = df_new.groupby('video')['change'].cumsum().astype(bool)

df_new = df_new[['ID', 'Driver_State_Changed']]
df_new.to_csv('../submissions/results_13122024_driver_of-bboxsize-ensemble.csv')